## Setup MySQL connection

In [78]:
import dotenv
import pandas as pd
import requests
from sqlalchemy.orm import Session

from persist.mysql import *
# import to create table
from persist.notam import *

dotenv.load_dotenv("../.env")

True

In [73]:
engine = MySQLEngine()

2023-05-13 12:23:05,629 INFO sqlalchemy.engine.Engine SELECT DATABASE()


INFO:sqlalchemy.engine.Engine:SELECT DATABASE()


2023-05-13 12:23:05,630 INFO sqlalchemy.engine.Engine [raw sql] {}


INFO:sqlalchemy.engine.Engine:[raw sql] {}


2023-05-13 12:23:07,201 INFO sqlalchemy.engine.Engine SELECT @@sql_mode


...

INFO:sqlalchemy.engine.Engine:DESCRIBE `notam_ai`.`notam_tag`


2023-05-13 12:23:07,658 INFO sqlalchemy.engine.Engine [raw sql] {}


INFO:sqlalchemy.engine.Engine:[raw sql] {}


2023-05-13 12:23:07,762 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [ ]:
# create the tables
Base.metadata.create_all(engine.get_engine())

## Get the NOTAMS from API

In [ ]:
locations = "ELLX,LFPG,EHAM"

notam_url = f"http://api.anbdata.com/anb/states/notams/notams-realtime-list?api_key={os.getenv('NOTAM_API_KEY')}&locations={locations}"

In [ ]:
res = requests.get(notam_url).json()
len(res)

In [ ]:
top20 = res[:20]
notams = [NOTAM(notam_id=notam['id'],
                message=notam['message'],
                location=notam['location']) for notam in top20]

In [ ]:
print(f'notam_id: {notams[1].notam_id}\n'
      f'message: {notams[1].message}\n'
      f'location: {notams[1].location}')

## Persist the NOTAMs to MySQL

In [ ]:
with Session(engine.get_engine()) as session:
    session.add_all(notams)

In [69]:
with Session(engine.get_engine()) as session:
    notam = session.query(NOTAM).first()
    print(notam)

2023-05-13 12:17:43,228 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-05-13 12:17:43,231 INFO sqlalchemy.engine.Engine SELECT notam.id AS notam_id_1, notam.notam_id AS notam_notam_id, notam.message AS notam_message, notam.location AS notam_location, notam.tag AS notam_tag, notam.summary AS notam_summary, notam.create_time AS notam_create_time, notam.update_time AS notam_update_time 
FROM notam 
 LIMIT %(param_1)s


INFO:sqlalchemy.engine.Engine:SELECT notam.id AS notam_id_1, notam.notam_id AS notam_notam_id, notam.message AS notam_message, notam.location AS notam_location, notam.tag AS notam_tag, notam.summary AS notam_summary, notam.create_time AS notam_create_time, notam.update_time AS notam_update_time 
FROM notam 
 LIMIT %(param_1)s


2023-05-13 12:17:43,232 INFO sqlalchemy.engine.Engine [cached since 128s ago] {'param_1': 1}


INFO:sqlalchemy.engine.Engine:[cached since 128s ago] {'param_1': 1}


<NOTAM(id=1, notam_id=V0230/23, message=[US DOD PROCEDURAL NOTAM] INSTRUMENT APPROACH PROCEDURE AMENDMENT
 RNP RWY 24 INCREASE LAND MINIMA. RAISE LNAV/VNAV DA** CAT D TO
 1617/1700M 404 (500-1700M). CHG * NO-LIGHT NOTE TO READ: **WHEN ALS
 INOP, INCR CAT ABC RVR/VIS TO 1700M; CAT D RVR/VIS TO 1900M.
CREATED: 13 Apr 2023 11:21:00 
SOURCE: KQZC, location=ELLX, tag=, summary=, create_time=2023-05-13 01:05:23, update_time=2023-05-13 01:05:23)>
2023-05-13 12:17:43,346 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


## Process the CSV Tags and persist to MySQL

In [64]:
tags = pd.read_csv('../data/tags.csv')
tags.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NOTAM TAGS (Final version for Sprint),NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,Description
3,NaN,NaN,NaN,NaN,Now optimised for AI
4,NaN,NaN,NaN,NaN,NaN
5,NaN,C,ATC,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,C1,ATC status/hours,NaN,"ATC operating hours, ATC Strike, ATC failure (..."
8,NaN,C2,ATC procedure,NaN,"TWR/APP/ACC change of procedure, lost comms pr..."
9,NaN,C3,ATC flow and delay,NaN,"Flow control, enroute delays, expect holding"


In [65]:
tags.drop(columns=[tags.columns[0], tags.columns[3]], axis=1, inplace=True)
tags.dropna(how='any', inplace=True)
tags

,Unnamed: 1,Unnamed: 2,Unnamed: 4
7,C1,ATC status/hours,"ATC operating hours, ATC Strike, ATC failure (..."
8,C2,ATC procedure,"TWR/APP/ACC change of procedure, lost comms pr..."
9,C3,ATC flow and delay,"Flow control, enroute delays, expect holding"
10,C4,Radio,"HF, VHF, CPDLC, Satcom, ATIS - u/s, freq chang..."
11,C5,Radar & ADS,"Radar (PSR, MSSR, SMR, PAR, TAR), ADS (ADS-B, ..."
12,C6,Met,"Met service hours, VOLMET, Met Equipment, Met ..."
17,P1,Airport status/hours,"Airport Closed, Airport operating hours, AD AP..."
18,P2,Airport restriction,"Not available as alternate, airport slots, PPR..."
19,P3,Fire & Rescue,"RFF Category change, Rescue equipment"
20,P4,Fuel,"All fuel related NOTAMs, JET, JETA1, Avgas, Hy..."


In [ ]:
# build notam tags model to persist
NOTAMTags = []
for index, row in tags.iterrows():
    print(row[0], row[1], row[2])
    NOTAMTags.append(NOTAMTag(level=row[0], tag=row[1], description=row[2]))

In [ ]:
# save the tags to MySQL
with Session(engine.get_engine(), expire_on_commit=False) as session:
    session.add_all(NOTAMTags)
    session.commit()

In [79]:
with Session(engine.get_engine()) as session:
    tag = session.query(NOTAMTag).first()
    print(tag)

2023-05-13 12:25:08,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-05-13 12:25:08,406 INFO sqlalchemy.engine.Engine SELECT notam_tag.id AS notam_tag_id, notam_tag.level AS notam_tag_level, notam_tag.tag AS notam_tag_tag, notam_tag.description AS notam_tag_description, notam_tag.class_name AS notam_tag_class_name 
FROM notam_tag 
 LIMIT %(param_1)s


INFO:sqlalchemy.engine.Engine:SELECT notam_tag.id AS notam_tag_id, notam_tag.level AS notam_tag_level, notam_tag.tag AS notam_tag_tag, notam_tag.description AS notam_tag_description, notam_tag.class_name AS notam_tag_class_name 
FROM notam_tag 
 LIMIT %(param_1)s


2023-05-13 12:25:08,407 INFO sqlalchemy.engine.Engine [cached since 113.9s ago] {'param_1': 1}


INFO:sqlalchemy.engine.Engine:[cached since 113.9s ago] {'param_1': 1}


2023-05-13 12:25:08,520 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK
